In [1]:
import pandas as pd
import numpy as np
import io

# --- 1. Simulate "RealEstate_Prices.csv" ---
# This file has messy column names, missing data, and outliers
csv_data = """Property ID,Location (Neighborhood),Property Type,Square Footage,Num Bedrooms,"Year Built","Sale Price"
1001,Downtown,Condo,850,2,2015,300000
1002,Suburbia,Single-Family,1600,3,1998,450000
1003,Uptown,Condo,1100,2,2010,380000
1004,Downtown,Apartment,,1,2005,250000
1005,Suburbia,Single-Family,1800,4,2005,510000
1006,Downtown,Condo,900,2,2015,310000
1007,Uptown,Townhouse,1300,3,2012,420000
1008,Suburbia,Single-Family,2100,4,2018,580000
1009,Downtown,Apartment,700,1,2005,260000
1010,Suburbia,,1900,4,2005,530000
1011,Uptown,Condo,3000,5,2018,15000000
"""

# --- 2. Simulate "Neighborhood_Amenities.csv" ---
# A second file to be merged
amenities_data = """Neighborhood,School_Rating,Park_Count,Crime_Rate
Downtown,7,5,0.45
Suburbia,9,12,0.20
Uptown,8,8,0.30
Rural,6,3,0.15
"""

# -------------------------------------------------------------
# ----------- DATA WRANGLING PIPELINE STARTS HERE -------------
# -------------------------------------------------------------

# Task 1: Import dataset and clean column names
# In a real case: df = pd.read_csv("RealEstate_Prices.csv")
df = pd.read_csv(io.StringIO(csv_data))
print("--- 1. Initial Column Names ---")
print(df.columns)

# Clean names: lowercase, replace spaces/special chars with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace(r'[^a-z0-9_]', '', regex=True)
print("\n--- 1. Cleaned Column Names ---")
print(df.columns)


# Task 2: Handle missing values
print("\n--- 2. Missing Values (Before) ---")
print(df.isnull().sum())

# Strategy 1: Impute numerical 'square_footage' with the median
median_sqft = df['square_footage'].median()
df['square_footage'] = df['square_footage'].fillna(median_sqft)

# Strategy 2: Impute categorical 'property_type' with the mode (most common)
mode_prop_type = df['property_type'].mode()[0]
df['property_type'] = df['property_type'].fillna(mode_prop_type)

print("\n--- 2. Missing Values (After) ---")
print(df.isnull().sum())


# Task 3: Perform data merging
# In a real case: amenities_df = pd.read_csv("Neighborhood_Amenities.csv")
amenities_df = pd.read_csv(io.StringIO(amenities_data))
amenities_df.columns = amenities_df.columns.str.lower() # Clean new df's columns too

print(f"\n--- 3. Merging Data ---")
print(f"Shape before merge: {df.shape}")

# Rename the 'neighborhood' column in amenities_df to match 'location_neighborhood' in df
amenities_df.rename(columns={'neighborhood': 'location_neighborhood'}, inplace=True)


# Perform a 'left' merge to keep all properties, even if they have no amenity data
df_merged = pd.merge(df, amenities_df, on='location_neighborhood', how='left')

print(f"Shape after merge: {df_merged.shape}")
print(df_merged.head())


# Task 7: Identify and handle outliers (Best done before aggregation)
print(f"\n--- 7. Handling Outliers ---")
print("Sale Price statistics (Before):")
print(df_merged['sale_price'].describe())

# Strategy: Cap extreme values using the IQR (Interquartile Range) method
Q1 = df_merged['sale_price'].quantile(0.25)
Q3 = df_merged['sale_price'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + (1.5 * IQR)
lower_bound = Q1 - (1.5 * IQR)

# Cap the values
df_merged['sale_price_capped'] = np.clip(df_merged['sale_price'], lower_bound, upper_bound)

print("\nSale Price statistics (After capping):")
print(df_merged[['sale_price', 'sale_price_capped']].describe())
# Note the original 'sale_price' (max 15M) vs 'sale_price_capped' (max ~685k)


# Task 4: Filter and subset the data
print(f"\n--- 4. Filtering Data ---")
# Criteria: Single-Family homes built after 2000
filtered_df = df_merged[
    (df_merged['property_type'] == 'Single-Family') &
    (df_merged['year_built'] > 2000)
].copy() # .copy() avoids a SettingWithCopyWarning

print("Filtered subset (Single-Family, built > 2000):")
print(filtered_df[['property_id', 'location_neighborhood', 'property_type', 'year_built', 'sale_price']])


# Task 6: Aggregate the data
print(f"\n--- 6. Aggregating Data ---")
# Calculate summary statistics by neighborhood
# We use the original 'sale_price' here to show the impact of the outlier
agg_stats = df_merged.groupby('location_neighborhood').agg(
    Average_Price=('sale_price', 'mean'),
    Average_Price_Capped=('sale_price_capped', 'mean'),
    Median_SqFt=('square_footage', 'median'),
    Average_School_Rating=('school_rating', 'mean'),
    Property_Count=('property_id', 'count')
).reset_index()

print("Average stats by neighborhood:")
print(agg_stats.to_string())
# Note the huge difference between Average_Price and Average_Price_Capped for 'Uptown'


# Task 5: Handle categorical variables
print(f"\n--- 5. Encoding Categorical Variables ---")
# We will create a new DataFrame for a "model-ready" state
# We use the capped price and drop the original
model_df = df_merged.drop(['sale_price', 'property_id'], axis=1).copy()

# Identify categorical columns for encoding
categorical_cols = ['location_neighborhood', 'property_type']

# Strategy: One-Hot Encoding
# drop_first=True helps prevent multicollinearity in linear models
model_df_encoded = pd.get_dummies(model_df, columns=categorical_cols, drop_first=True)

print("DataFrame head after One-Hot Encoding:")
print(model_df_encoded.head())
print("\nNew columns created by encoding:")
print(model_df_encoded.columns)

--- 1. Initial Column Names ---
Index(['Property ID', 'Location (Neighborhood)', 'Property Type',
       'Square Footage', 'Num Bedrooms', 'Year Built', 'Sale Price'],
      dtype='object')

--- 1. Cleaned Column Names ---
Index(['property_id', 'location_neighborhood', 'property_type',
       'square_footage', 'num_bedrooms', 'year_built', 'sale_price'],
      dtype='object')

--- 2. Missing Values (Before) ---
property_id              0
location_neighborhood    0
property_type            1
square_footage           1
num_bedrooms             0
year_built               0
sale_price               0
dtype: int64

--- 2. Missing Values (After) ---
property_id              0
location_neighborhood    0
property_type            0
square_footage           0
num_bedrooms             0
year_built               0
sale_price               0
dtype: int64

--- 3. Merging Data ---
Shape before merge: (11, 7)
Shape after merge: (11, 10)
   property_id location_neighborhood  property_type  square_foota